In [2]:
import librosa
import numpy as np
from CQT import cqt

In [3]:
y, sr = librosa.load('data/MusicDelta_Hendrix_STEM_04.wav')
C = np.abs(librosa.cqt(y, sr=16000, fmin=librosa.note_to_hz('C1')
                       ,n_bins=24)) # n_bins upto 113


In [5]:
C_ = np.abs(cqt(y, fmin=librosa.note_to_hz('A1'), fmax=librosa.note_to_hz('A8'), bins=128, fs=sr)['spCQT'])

AttributeError: module 'numpy' has no attribute 'complex'.
`np.complex` was a deprecated alias for the builtin `complex`. To avoid this error in existing code, use `complex` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.complex128` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [4]:
y.shape
C.shape

(24, 855)